<a href="https://colab.research.google.com/github/JohatTech/LSTM-recurrent-neural-network-arquitecture/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LSTM neural network arquitecture
this is the my first project of recurrent neural network.

In this project I will build a LSTM arquitecture to identify basic emotions in amazon products reviews both positive(happy, excited, etc.) and negative (sad, worried, etc.).

Importing the basic libraries

In [36]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
import numpy as np 
import pandas as pd 
import re
import keras
from keras import Model
from keras.layers import Flatten, LSTM,  Dense, Embedding
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras_preprocessing.text import Tokenizer
from keras.initializers import glorot_uniform
from sklearn import model_selection

# **Loading the data**
The dataset I will use is the amazon product review that is stored on the tensorflow database.

Then I will convert to a pandas dataframe.


In [37]:
import tensorflow_datasets as tdfs
data = tdfs.load('amazon_us_reviews/Mobile_Electronics_v1_00', split='train', shuffle_files=True)
assert isinstance(data, tf.data.Dataset)#check is the dataset still exist in the dataset 
print(data)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


<_OptionsDataset element_spec={'data': {'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'helpful_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'marketplace': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_category': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_parent': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_body': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_date': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_headline': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'star_rating': TensorSpec(shape=(), dtype=tf.int32, name=None), 'total_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'verified_purchase': TensorSpec(shape=(), dtype=tf.int64, name=None), 'vine': TensorSpec(shape=(), dtype=

In [38]:
df = tdfs.as_dataframe(data)

In [39]:
df['sentiment'] =  df['data/star_rating'].apply(lambda score: "positive" if score >=3 else "negative")

df['sentiment'] = df['sentiment'].map({'positive':1, "negative":0})

df['short_review'] =df['data/review_body'].str.decode("utf-8")
df = df[["short_review", "sentiment"]]

In [40]:
# Dropping last n rows using drop
n = 54975
df.drop(df.tail(n).index,
        inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# **Tokenization of the text and convertion into tokens**
Since the data is about text, machine learning models understand numberical features, I need to convert the review columns into numerical values. 

In this application,I'm going to use the Tokenizer class from Keras preprocessing

The tokenizer will convert each word in a sentence into integer tokens/IDs based on the frequency of the word appearing in the corpus.

I will convert our feature column and label into a set of lists as that’s how our Tokenizer wants our data. To apply the Tokenizer to the corpus, I will split the dataset into training and testing sets as the tokenizer only needs to be fit on the training set and not on the test set.

In [41]:
#split data into 70% train and 30% test
x_train, x_test, y_train, y_test = model_selection.train_test_split(df['short_review'],df['sentiment'], test_size=0.30)

In [42]:
#converting to array 

x_train = np.array(x_train.values.tolist())
x_test = np.array(x_test.values.tolist())
y_train = np.array(y_train.values.tolist())
y_test = np.array(y_test.values.tolist())

In [43]:
#applying tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
total_size =  len(word_index)+1
print(total_size)

33806


In [44]:
#text to sequences

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
#add padding to ensure the same length
max_length = 100
x_train = pad_sequences(x_train, padding='post', maxlen=max_length)
x_test = pad_sequences(x_test, padding='post', maxlen=max_length)

Now I going to start building the model, since the goal is to build a LSTM arquitecture, the structure of the model is as follow:



1.  The Keras embedding layer will be trained with the word that we get from the reviews, initializing with random weights and is defined as the first hidden layers of the network
2.   EThe LSTM layer are going to learn what emotion are represented int the words if is negative or positive.
3.   The last one is the Dense layer which is the fully connected layer that ensure that the model learn well and improve his accuracy.



In [45]:

model = Sequential()
model.add(Embedding(total_size, 20, input_length=100))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation ='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 20)           676120    
                                                                 
 lstm_4 (LSTM)               (None, 32)                6784      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 682,937
Trainable params: 682,937
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


274/274 [==============================] - 318s 1s/step - loss: 0.5591 - acc: 0.7597 - val_loss: 0.5437 - val_acc: 0.7619
Epoch 2/5
274/274 [==============================] - 296s 1s/step - loss: 0.5139 - acc: 0.7791 - val_loss: 0.4119 - val_acc: 0.8467
Epoch 3/5
274/274 [==============================] - 291s 1s/step - loss: 0.4502 - acc: 0.8195 - val_loss: 0.4589 - val_acc: 0.8143
Epoch 4/5
274/274 [==============================] - 292s 1s/step - loss: 0.4570 - acc: 0.8059 - val_loss: 0.4550 - val_acc: 0.8062
Epoch 5/5
274/274 [==============================] - 335s 1s/step - loss: 0.4357 - acc: 0.8221 - val_loss: 0.4583 - val_acc: 0.8338


# **Performance analysis**

after trainning the model with 5 epochs and 128 batches I got a loss of 43% and a accuracy of 82.1% on the training side and a loss of 45%  and accuracy of 83.3% in the validation data, which is a great indicator of good performance


# **Conclusion**

That's it for the project, in this case I better understand now the arquitecture of LSTM models and the advantage tha this have over sequencial data, this model need more hiper-parameter tunning and testing with real-life examples from customer reviews, also we can also compare the this model, with other NLP tools like transformer for example, but this will be in a future project. you can download the H5 file in the GitHub repository to use the pre-trained model to futher analysis.

In [47]:
model.save('model.h5')